# Compare ACE2 affinity across different experiments

First, get parameters specifying data to analyze:

In [ ]:
# this cell is tagged as `parameters` for papermill parameterization
xbb_spike_affinity = None
xbb_spike_func_effects = None
xbb_spike_escape = None

starr_rbd_affinity = None

ba2_spike_affinity = None
ba2_spike_func_effects = None

xbb_rbd_affinity_monomeric = None
xbb_rbd_affinity_dimeric = None
xbb_rbd_func_effects = None

merged_affinity_csv = None

In [ ]:
import os

import altair as alt

import pandas as pd

## Read the data sets

In [ ]:
xbb_spike = pd.read_csv(xbb_spike_csv).assign(
    experiment="XBB.1.5 full spike pseudovirus",
    ACE2_type="monomeric",
)

ba2_spike = pd.read_csv(ba2_spike_csv).assign(
    experiment="BA.2 full spike pseudovirus",
    ACE2_type="monomeric",
)

xbb_rbd = (
    pd.read_csv(xbb_rbd_csv)
    .rename(
        columns={
            f"{ace2_type} ACE2 affinity": ace2_type
            for ace2_type in ["monomeric", "dimeric"]
        }
    )
    .melt(
        id_vars=["site", "wildtype", "mutant", "spike mediated entry", "sequential_site", "region"],
        var_name="ACE2_type",
        value_name="ACE2 affinity",
        value_vars=["monomeric", "dimeric"],
    )
    .assign(experiment="XBB.1.5 RBD pseudovirus")
)

starr_rbd = (
    pd.read_csv(starr_rbd_affinity)
    .rename(columns={"position": "site", "delta_bind": "ACE2 affinity", "target": "experiment"})
    .query("experiment in ['Omicron_BA2', 'Omicron_XBB15']")
    .assign(
        experiment=lambda x: x["experiment"].map(
            {
                "Omicron_BA2": "BA.2 RBD yeast display",
                "Omicron_XBB15": "XBB.1.5 RBD yeast display",
            }
        )
    )
    .assign(ACE2_type="monomeric", region="RBD")
    [["site", "wildtype", "mutant", "ACE2 affinity", "region", "experiment", "ACE2_type"]]
)

merged_affinity_data = pd.concat([xbb_spike, ba2_spike, xbb_rbd, starr_rbd], ignore_index=True)

print(f"Writing the merged affinity data to {merged_affinity_csv}")
os.makedirs(os.path.dirname(merged_affinity_csv), exist_ok=True)
merged_affinity_data.to_csv(merged_affinity_csv, float_format="%.4f", index=False)